In [1]:
from statsmodels.stats.multitest import multipletests
import scrublet as scr
import scanpy as sc
import pandas as pd
import numpy as np
import scipy
import os

sc.settings.verbosity = 1

#some functions that Ni uses in scanpy scripts to run scrublet
#which in turn are inspired by my original notebook on the matter
#(extracted from scanpy_scripts 0.2.10 to get around scanpy version incompatibility)
def test_outlier(x, upper_mad_only=True):
	med = np.median(x)
	if upper_mad_only:
		mad = np.median(x[x>med] - med) * 1.4826
	else:
		mad = np.median(np.abs(x - med)) * 1.4826
	pvals = 1 - scipy.stats.norm.cdf(x, loc=med, scale=mad)
	bh_pvals = multipletests(pvals, method='fdr_bh')[1]
	return pvals, bh_pvals

def run_scrublet(adata, resolution_function=None):
	old_verbosity = sc.settings.verbosity
	sc.settings.verbosity = 1
	if resolution_function is None:
		resolution_function = lambda x: np.maximum(np.maximum(np.log10(x)-1, 0)**2, 0.1)
	scrub = scr.Scrublet(adata.X)
	#this has the potential to brick for poor quality data
	#if so, abort it and everything downstream
	try:
		ds, pd = scrub.scrub_doublets(verbose=False)
	except:
		return
	adata.obs['scrublet_score'] = ds

	adata_copy = adata.copy()
	sc.pp.filter_genes(adata_copy, min_cells=3)
	sc.pp.normalize_total(adata_copy, target_sum=1e4)
	sc.pp.log1p(adata_copy)
	sc.pp.highly_variable_genes(adata_copy, min_mean=0.0125, max_mean=3, min_disp=0.5, subset=True)
	sc.pp.scale(adata_copy, zero_center=False)
	sc.pp.pca(adata_copy, svd_solver='arpack', zero_center=False)
	sc.pp.neighbors(adata_copy, n_pcs=30)
	sc.tl.umap(adata_copy)
	sc.tl.leiden(adata_copy, resolution=1)
	for clst in np.unique(adata_copy.obs['leiden']):
		clst_size = sum(adata_copy.obs['leiden'] == clst)
		sc.tl.leiden(adata_copy, restrict_to=('leiden', [clst]), resolution=resolution_function(clst_size), key_added='leiden_R')
		adata_copy.obs['leiden'] = adata_copy.obs['leiden_R']
	clst_meds = []
	for clst in np.unique(adata_copy.obs['leiden']):
		k = adata_copy.obs['leiden'] == clst
		clst_med = np.median(adata_copy.obs.loc[k, 'scrublet_score'])
		adata_copy.obs.loc[k, 'cluster_scrublet_score'] = clst_med
		clst_meds.append(clst_med)
	clst_meds = np.array(clst_meds)
	pvals, bh_pvals = test_outlier(clst_meds)
	for i, clst in enumerate(np.unique(adata_copy.obs['leiden'])):
		k = adata_copy.obs['leiden'] == clst
		adata_copy.obs.loc[k, 'pval'] = pvals[i]
		adata_copy.obs.loc[k, 'bh_pval'] = bh_pvals[i]
	sc.settings.verbosity = old_verbosity
	#need to also export the clustering, for soupx purposes
	adata.obs['scrublet_leiden'] = adata_copy.obs['leiden']
	adata.obs['scrublet_score'] = adata_copy.obs['scrublet_score']
	adata.obs['cluster_scrublet_score'] = adata_copy.obs['cluster_scrublet_score']
	adata.obs['doublet_pval'] = adata_copy.obs['pval']
	adata.obs['doublet_bh_pval'] = adata_copy.obs['bh_pval']
	del adata_copy

Pipeline scrublet above, list of samples below:

In [2]:
samples = ['Pan_T7917815','Pan_T7917816','Pan_T7917817','Pan_T7917818','Pan_T7917819','Pan_T7917820','Pan_T7917821','Pan_T7917822','Pan_T7917823','Pan_T7917824','Pan_T7917826','Pan_T7917827','Pan_T7935487','Pan_T7935488','Pan_T7935489','Pan_T7935490','Pan_T7935490','Pan_T7935491','Pan_T7935492','Pan_T7935493','Pan_T7935494','Pan_T7935495','Pan_T7935496','Pan_T7935497','Pan_T7980358','Pan_T7980359','Pan_T7980360','Pan_T7980361','Pan_T7980364','Pan_T7980365','Pan_T7980366','Pan_T7980367','Pan_T7980368','Pan_T7980369','Pan_T7980370','Pan_T7991584','Pan_T7991585','Pan_T7991586','Pan_T7991587','Pan_T7991588','Pan_T7991589','Pan_T7991590','Pan_T7991591','Pan_T7991592','Pan_T7991593','Pan_T7991594','Pan_T7991595','Pan_T8010318','Pan_T8010319','Pan_T8010320','Pan_T8010321','Pan_T8010323','Pan_T8010325','Pan_T8010326','Pan_T8010327','Pan_T8010330','Pan_T8010331','Pan_T8986827','Pan_T8986828','Pan_T8986829','Pan_T8986830','Pan_T8986831']

Run the pipeline scrublet for all the samples.

In [3]:
for sample in samples:
    print(sample)
    adata = sc.read_10x_h5('/warehouse/cellgeni/tic-kp9-cc36/'+sample+'/outs/filtered_feature_bc_matrix.h5')
    adata.var_names_make_unique()
    #run Ni's adaptation of my adaptation of scrublet
    run_scrublet(adata)
    #more anti-brick protection - the scores won't show up in the object if scrublet fails
    if 'scrublet_score' in adata.obs.columns:
        adata.obs[['scrublet_leiden', 'scrublet_score', 'cluster_scrublet_score', 'doublet_pval', 'doublet_bh_pval']].reset_index().to_csv(sample+'.csv', index=False)

Pan_T7917815


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Pan_T7917816


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Pan_T7917817
Pan_T7917818


...

Pan_T8986829


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Pan_T8986830


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Pan_T8986831


Don't worry about those desynchronised variable name warnings. That's just the notebook being wonky in printing stuff in order.

Run irods samples. List acquired from Cecilia.

In [5]:
samples = ['CZI-IA9924320+CZI-IA9924368',
 'CZI-IA9924321+CZI-IA9924369',
 'CZI-IA10244332+CZI-IA10244434',
 'CZI-IA10244331+CZI-IA10244433',
 'CZI-IA10466281+CZI-IA10466377',
 'CZI-IA10466282+CZI-IA10466378',
 'CZI-IA10466283+CZI-IA10466379',
 'CZI-IA10466284+CZI-IA10466380',
 'CZI-IA10466285+CZI-IA10466381',
 'CZI-IA10466286+CZI-IA10466382',
 'CZI-IA10471909+CZI-IA10472005',
 'CZI-IA10471910+CZI-IA10472006',
 'CZI-IA10471911+CZI-IA10472007',
 'CZI-IA10471912+CZI-IA10472008',
 'CZI-IA10471913+CZI-IA10472009']

for sample in samples:
    print(sample)
    #this time we download the count matrix off irods (and delete after loading)
    os.system('irods iget -K /archive/HCA/10X/'+sample+'/outs/filtered_feature_bc_matrix.h5')
    adata = sc.read_10x_h5('filtered_feature_bc_matrix.h5')
    os.system('rm filtered_feature_bc_matrix.h5')
    adata.var_names_make_unique()
    #run Ni's adaptation of my adaptation of scrublet
    run_scrublet(adata)
    #more anti-brick protection - the scores won't show up in the object if scrublet fails
    if 'scrublet_score' in adata.obs.columns:
        adata.obs[['scrublet_leiden', 'scrublet_score', 'cluster_scrublet_score', 'doublet_pval', 'doublet_bh_pval']].reset_index().to_csv(sample+'.csv', index=False)

CZI-IA9924320+CZI-IA9924368


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


CZI-IA9924321+CZI-IA9924369


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


CZI-IA10244332+CZI-IA10244434


...

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


CZI-IA10471912+CZI-IA10472008


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


CZI-IA10471913+CZI-IA10472009


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


And for my next trick, New York samples.

In [6]:
samples = ['CZINY-0049','CZINY-0050','CZINY-0051','CZINY-0052','CZINY-0053','CZINY-0054','CZINY-0055','CZINY-0056','CZINY-0057','CZINY-0058','CZINY-0059','CZINY-0060','CZINY-0061','CZINY-0062','CZINY-0063','CZINY-0064','CZINY-0097','CZINY-0098','CZINY-0099','CZINY-0100','CZINY-0101','CZINY-0102','CZINY-0103','CZINY-0104','CZINY-0105','CZINY-0106','CZINY-0107','CZINY-0108','CZINY-0109','CZINY-0110','CZINY-0111','CZINY-0112','CZINY-0161','CZINY-0162','CZINY-0163','CZINY-0164','CZINY-0165','CZINY-0166','CZINY-0167','CZINY-0168','CZINY-0169','CZINY-0170','CZINY-0171','CZINY-0172','CZINY-0173','CZINY-0174','CZINY-0175','CZINY-0176','CZINY-0225','CZINY-0226','CZINY-0227','CZINY-0228','CZINY-0229','CZINY-0230','CZINY-0231','CZINY-0232','CZINY-0233','CZINY-0234','CZINY-0235','CZINY-0236','CZINY-0237','CZINY-0238','CZINY-0239','CZINY-0240']

for sample in samples:
    print(sample)
    adata = sc.read_10x_h5('/lustre/scratch117/cellgen/team205/sharedData/cc36/ny-cites/'+sample+'/outs/filtered_feature_bc_matrix.h5')
    adata.var_names_make_unique()
    #run Ni's adaptation of my adaptation of scrublet
    run_scrublet(adata)
    #more anti-brick protection - the scores won't show up in the object if scrublet fails
    if 'scrublet_score' in adata.obs.columns:
        adata.obs[['scrublet_leiden', 'scrublet_score', 'cluster_scrublet_score', 'doublet_pval', 'doublet_bh_pval']].reset_index().to_csv(sample+'.csv', index=False)

CZINY-0049


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


CZINY-0050


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


CZINY-0051


...

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


CZINY-0239


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


CZINY-0240


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
